# Laboratory of Computational Physics mod. B
# Title????
Authors:
Valeria Fioroni, 
Matteo Guida,
Philipp Zehetner

## Scattered notes
We have the the total number of events are:
$$N = \int L \, dt \cdot \sigma$$


In [1]:
import pandas as pd
import os
from root_pandas import read_root
import tensorflow as tf
import seaborn as sns
import numpy as np

Welcome to JupyROOT 6.16/00


# Read Data

In [2]:
#Read background and signal samples from ROOT files
#BACKGROUND
'''CONTINUUM BACKGROUND'''
Continuum_bkg = pd.DataFrame() #empty dataframe to contain continuum bkg samples
#export all continuum bkg root files to a single pandas dataframe
for file in os.listdir("../DataBelle2/Background/Continuum/"):
    continuum_temp = pd.DataFrame()
    continuum_temp = read_root("../DataBelle2/Background/Continuum/" + file, 'B0')
    frames=[Continuum_bkg, continuum_temp]
    Continuum_bkg= pd.concat(frames, ignore_index=True)
    
'''PEAKING BACKGROUND'''
Peaking_bkg = pd.DataFrame() #empty dataframe to contain peaked bkg samples
#export all peaking bkg root files to a single pandas dataframe
for file in os.listdir("../DataBelle2/Background/Peaking/"):
    peaking_temp = pd.DataFrame()
    peaking_temp = read_root("../DataBelle2/Background/Peaking/" + file, 'B0')
    frames=[Peaking_bkg, peaking_temp]
    Peaking_bkg= pd.concat(frames, ignore_index=True)    


'''SIGNAL'''
Signal = pd.DataFrame() #empty dataframe to contain signal samples
#export all peaking bkg root files to a single pandas dataframe
for file in os.listdir("../DataBelle2/Signal/"):
    signal_temp = pd.DataFrame()
    signal_temp = read_root("../DataBelle2/Signal/" + file, 'B0')
    frames=[Signal, signal_temp]
    Signal= pd.concat(frames, ignore_index=True) 


#Signal: require B0_isSignal == True
Signal=Signal[Signal['B0_isSignal']== True]
print(Signal.shape)

#Continuum: require B0_isContinuumEvent == True
Continuum_bkg=Continuum_bkg[Continuum_bkg['B0_isContinuumEvent']== True]
print(Continuum_bkg.shape)

#Peaking: require B0_isNotContinuumEvent == True
Peaking_bkg=Peaking_bkg[Peaking_bkg['B0_isNotContinuumEvent']== True]
print(Peaking_bkg.shape)


'''DNN Error: Input contains NaN, infinity or a value too large for dtype('float64')'''
#Check if there are NaN in datasets (see below why)
Peaking_bkg.replace([np.inf, -np.inf], np.nan)
Continuum_bkg.replace([np.inf, -np.inf], np.nan)
Signal.replace([np.inf, -np.inf], np.nan)
N_peak=Peaking_bkg.isnull().T.any().T.sum()
print(N_peak)
N_cont=Continuum_bkg.isnull().T.any().T.sum()
print(N_cont)
N_sig=Signal.isnull().T.any().T.sum()
print(N_sig)


Important_variables=[
"B0_Pcms","B0_P4cms",
"B0_etap_Pcms" ,
"B0_etap_P4cms",
"B0_etap_eta_gamma0_Pcms",
"B0_etap_eta_gamma0_P4cms",   
"B0_etap_eta_gamma1_Pcms",  
"B0_etap_eta_gamma1_P4cms",            
"B0_etap_pi1_Pcms",                     
"B0_K_S0_P4cms",
"B0_K_S0_M",                
"B0_M",          
"B0_etap_M",     
"B0_etap_eta_M",
"B0_TrCSMVA"]

/home/utente/.local/lib/python3.6/site-packages/root_pandas/readwrite.py:320: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  df = df.reindex_axis(columns, axis=1, copy=False)


(20491, 391)
(1847, 391)
(208, 391)
0
0
0


In [3]:
#Select variables useful for the Classification task
Signal_selection = Signal[Important_variables]
Continuum_bkg_selection= Continuum_bkg[Important_variables]
Peaking_bkg_selection= Peaking_bkg[Important_variables]

print(Continuum_bkg_selection.shape)
print(Peaking_bkg_selection.shape)
print(Signal_selection.shape)


#The Dataframe contains arrays as elements, I create a new column for each component
Signal_comp = pd.concat([pd.DataFrame(Signal_selection[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Signal_selection[c] for c in Signal_selection], axis=1)
Peaking_bkg_comp = pd.concat([pd.DataFrame(Peaking_bkg_selection[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Peaking_bkg_selection[c] for c in Peaking_bkg_selection], axis=1)
Continuum_bkg_comp = pd.concat([pd.DataFrame(Continuum_bkg_selection[c].tolist()).add_prefix(c[:c.index('4')]).add_suffix(c[c.index('4')+1:]) if '4' in c else Continuum_bkg_selection[c] for c in Continuum_bkg_selection], axis=1)

(1847, 15)
(208, 15)
(20491, 15)


In [4]:
from IPython.display import display
pd.options.display.max_columns = None
display(Continuum_bkg_comp)

,B0_Pcms,B0_P0cms,B0_P1cms,B0_P2cms,B0_P3cms,B0_etap_Pcms,B0_etap_P0cms,B0_etap_P1cms,B0_etap_P2cms,B0_etap_P3cms,B0_etap_eta_gamma0_Pcms,B0_etap_eta_gamma0_P0cms,B0_etap_eta_gamma0_P1cms,B0_etap_eta_gamma0_P2cms,B0_etap_eta_gamma0_P3cms,B0_etap_eta_gamma1_Pcms,B0_etap_eta_gamma1_P0cms,B0_etap_eta_gamma1_P1cms,B0_etap_eta_gamma1_P2cms,B0_etap_eta_gamma1_P3cms,B0_etap_pi1_Pcms,B0_K_S0_P0cms,B0_K_S0_P1cms,B0_K_S0_P2cms,B0_K_S0_P3cms,B0_K_S0_M,B0_M,B0_etap_M,B0_etap_eta_M,B0_TrCSMVA
0,0.669950,0.001286,0.441464,0.503926,5.186840,2.503328,-1.530360,1.772007,-0.885801,2.680545,0.384088,-0.037835,0.368723,-0.100675,0.384088,0.831146,-0.703095,0.392105,-0.206680,0.831146,0.904634,1.531646,-1.330543,1.389727,2.506295,0.483620,5.143392,0.941673,0.504473,-2.871097
1,0.335508,-0.019173,0.256334,0.215617,5.193409,2.510330,-1.550819,1.586877,-1.174111,2.687115,0.350659,-0.085040,0.142322,-0.308990,0.350659,0.831146,-0.703095,0.392105,-0.206680,0.831146,0.904634,1.531646,-1.330543,1.389727,2.506295,0.483620,5.182561,0.988096,0.473267,-1.168907
2,0.684248,0.311205,-0.235870,0.561883,5.463105,2.855099,1.375249,1.088634,2.252811,3.011410,1.175187,0.534053,0.611319,0.849788,1.175187,0.781588,0.392986,0.060522,0.672888,0.781588,0.172808,-1.064045,-1.324504,-1.690929,2.451696,0.514850,5.420086,0.936667,0.446823,-2.470485
3,0.746486,-0.078040,-0.218507,0.709511,5.115091,2.355849,-1.709389,1.615634,0.133186,2.543518,0.270169,-0.211519,0.102913,-0.132890,0.270169,1.432915,-0.887753,1.103418,0.218195,1.432915,0.338019,1.631349,-1.834141,0.576324,2.571574,0.505441,5.060328,0.980953,0.479150,-4.579593
4,0.871923,0.232268,0.319564,0.777291,5.117590,2.097735,-1.711981,0.232707,-1.189731,2.305975,0.819212,-0.479343,0.008932,-0.664274,0.819212,0.462176,-0.393852,0.241815,0.003538,0.462176,0.830864,1.944249,0.086857,1.967022,2.811614,0.498347,5.042764,0.956602,0.616474,-5.430493
5,0.589619,-0.333525,0.066233,0.481689,5.284344,2.309825,-0.327126,-1.142911,-1.980413,2.499248,1.216810,-0.126230,-0.490448,-1.106414,1.216810,0.057045,-0.015152,-0.027858,0.047418,0.057045,0.236236,-0.006399,1.209144,2.462102,2.785096,0.482434,5.251346,0.944707,0.461090,-4.223256
6,0.577331,-0.335425,-0.017741,0.469560,5.333735,2.362992,-0.329026,-1.226885,-1.992542,2.548639,1.216810,-0.126230,-0.490448,-1.106414,1.216810,0.127893,-0.034694,-0.116356,0.040178,0.127893,0.236236,-0.006399,1.209144,2.462102,2.785096,0.482434,5.302397,0.935810,0.526552,-3.859004
7,0.528301,-0.242748,-0.004663,0.469206,5.316834,2.345383,-0.236349,-1.213807,-1.992896,2.531739,1.216810,-0.126230,-0.490448,-1.106414,1.216810,0.134240,0.079128,-0.107958,-0.010204,0.134240,0.236236,-0.006399,1.209144,2.462102,2.785096,0.482434,5.290523,0.956052,0.467108,-4.040184
8,0.756288,0.513818,-0.288207,-0.474237,5.414255,2.538408,1.820456,1.240062,1.261625,2.713208,0.160581,0.143529,0.017554,-0.069839,0.160581,1.368135,1.031404,0.564757,0.699320,1.368135,0.577116,-1.306638,-1.528269,-1.735862,2.701047,0.489415,5.361174,0.969970,0.470291,-3.708413
9,0.742017,0.312194,-0.417885,-0.527728,5.197163,2.305029,1.618832,1.110384,1.208134,2.496115,0.074452,0.029659,-0.041885,-0.053936,0.074452,1.368135,1.031404,0.564757,0.699320,1.368135,0.577116,-1.306638,-1.528269,-1.735862,2.701047,0.489415,5.143919,0.943813,0.515060,-3.156705


###  We need to understand whether to put cuts on the dataset i.e. simply setting some if on the variables and if they are not respected eliminate the event.

## Step 2
I tried a feature scaling on the dataset.
Centered the points around their mean and scaled by their sample standard deviation. 
It seems that in general the algorithms of machine learning algorithms perform better after this kind of scaling.

In [5]:
from random import seed
from random import randint
# seed random number generator
n_seed=2347
seed(n_seed)

In [6]:
import sklearn
from sklearn.model_selection import train_test_split
Signal_comp['Type'] = 2 #Signal label == 2
Peaking_bkg_comp['Type'] = 1 # Peaking background label == 1
Continuum_bkg_comp['Type'] = 0 #Continuum backgroun label == 0
# Put in a dataframe randomly shuffled the lines
Sum_BS = pd.concat([Signal_comp,Peaking_bkg_comp,Continuum_bkg_comp]).sample(frac=1)
X = Sum_BS.drop('Type',axis=1)
Y = Sum_BS['Type']
# Dataframe divided into 80% of train data and 20% of test
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8,random_state=randint(10**6,10**9))
# Standardize features by removing the mean and scaling to unit variance

Standardization of a dataset is a common requirement for many machine learning estimators: 
they might behave badly if the individual features do not more or less look like standard normally 
distributed data (e.g. Gaussian with 0 mean and unit variance). [More infos](https://scikit-learn.org/stable/modules/preprocessing.html)

In [7]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
print(ss.fit(X_train))
# The averages and standard deviations of all the databases are saved, 
# now with transfer I modify the data (Train and Test) in practice
X_train = ss.transform(X_train)
X_test = ss.transform(X_test)

'''DNN Error: Input contains NaN, infinity or a value too large for dtype('float64')'''
#Check if there are NaN in X_train and X_test
Null_X_train=np.isnan(X_train)#.T.any().T.sum()
print(Null_X_train)
Null_X_test=np.isnan(X_test)#.T.any().T.sum()
print(Null_X_test)


StandardScaler(copy=True, with_mean=True, with_std=True)
[[False False False ... False False False]
 [ True False False ...  True  True  True]
 [False False False ... False False False]
 ...
 [False  True  True ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
[[False False False ... False False False]
 [False  True  True ... False False False]
 [ True False False ...  True  True  True]
 ...
 [False  True  True ... False False False]
 [False False False ... False False False]
 [ True False False ...  True  True  True]]


# Model Optimization

# ************** ancora da modificare **************
in "parameters" I would like to put also different possibilities for learning rate and solver at least. Maybe just 3 different learning rates and 2 different solvers, otherwise it could take too long to run

In [8]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [9]:
parameters = {'hidden_layer_sizes': [(100,),(100,100,),(100,100,100,), (200,300,100,), (100,200,300,100,)]}

mlp = MLPClassifier(max_iter=300, alpha=1e-4,
                    solver='sgd', tol=1e-4, random_state=n_seed,
                    learning_rate_init=.1)

clf= GridSearchCV(mlp,param_grid=parameters,verbose=2,cv=5,return_train_score=True)

clf.fit(X_train, Y_train)

print ('RESULTS FOR NN\n')
print("Best parameters set found:")
print(clf.best_params_)

print("Score with best parameters:")
print(clf.best_score_)

print("\nAll scores on the grid:")
scores=pd.DataFrame(clf.cv_results_)
print(scores[['param_hidden_layer_sizes','mean_test_score']])

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] hidden_layer_sizes=(100,) .......................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/home/utente/.local/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:528: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').